# FMZ Übung 6: **Klassifikationsalgorithmen**

---
### Ziel der Übung
- Vergleich verschiedener Klassifikationsalgorithmen (logistische Regression, linear Diskrimanzanalyse, Naive-Bayes-Klassifikator, Entscheidungsbaum, Random Forest, Support Vector Machine, neuronale Netze)
- Einsatz mehrerer Fehlermetriken (Accuracy, False Positive Rate, False Negative Rate, F1-Score)
- Übersichtliche Darstellung der Ergebnisse
- Anwendung auf unterschiedliche Datensätze
- Begriff von Kreuzvalidierung


---
### Benötigte Pakete

```{r, eval=FALSE}
 install.packages(c("randomForest","e1071","neuralnet","mlbench")
 library(ggplot2)
 library(randomForest)
 library(e1071)
 library(MASS)
 library(rpart)
 library(nnet)
 library(mlbench)
```
---


In [ ]:
install.packages(c("randomForest","e1071","neuralnet","mlbench"))
 library(ggplot2)
 library(randomForest)
 library(e1071)
 library(MASS)
 library(rpart)
 library(nnet)
 library(mlbench)

---
## Iris-Daten
Die Iris-Daten bestehen aus 150 Blumenmessungen dreier Iris-Arten, jeweils mit vier Merkmalen: Kelchblattlänge, Kelchblattbreite, Kronblattlänge und Kronblattbreite.

---
### Aufgabe 1: **Visualisierung der Daten (Scatterplot und Hauptkomponenanalyse)**

In [ ]:

# Scatterplot
plot(iris[-5], col=as.numeric(iris$Species)+1, pch=as.numeric(iris$Species)+15)

# PCA berechnen
pca <- prcomp(iris[-5], scale.=FALSE)
pca_df <- as.data.frame(pca$x[,1:2])
pca_df$Class <- iris$Species

# Mittelwerte pro Klasse berechnen
class_centers <- aggregate(cbind(PC1, PC2) ~ Class, data=pca_df, FUN=mean)

# Scatterplot der zwei ersten Komponenten
ggplot(pca_df, aes(PC1, PC2, color=Class)) +
  geom_point(size=2) +
  geom_point(data=class_centers, aes(PC1, PC2), color=c(2,3,4), size=7, shape=10) +
  labs(title="PCA Scatterplot: Iris-Daten (Klassen)") +
  theme_bw()


---
### Aufgabe 2: **Clusteranalyse**

In [ ]:

# K-means Clusteranalyse
set.seed(123)
km <- kmeans(iris[-5], centers = 3)
pca_df$Cluster <- factor(km$cluster)


# Mittelwerte pro Cluster berechnen
cluster_centers <- aggregate(cbind(PC1, PC2) ~ Cluster, data=pca_df, FUN=mean)

# Plots
ggplot(pca_df, aes(PC1, PC2, color=Cluster)) +
  geom_point(size=2) +
  geom_point(data=class_centers, aes(PC1, PC2), color=c(2,3,4), size=7, shape=10) +
  geom_point(data=cluster_centers, aes(PC1, PC2), color=c(2,3,4), size=7, shape=8) +
  labs(title="k-means Cluster (k=3) auf PCA-Koordinaten") +
  theme_bw()


### Aufgabe 3: **Training der Klassifikationsalgorithmen**

Logistische Regression, linear Diskrimanzanalyse, Naive-Bayes-Klassifikator, Entscheidungsbaum, Random Forest, Support Vector Machine, neuronale Netze

In [ ]:

# Algorithmen auf dem kompletten Datensatz trainieren
m_log  <- multinom(Species ~ ., data=iris, trace=FALSE)
m_lda  <- lda(Species ~ ., data=iris)
m_nb   <- naiveBayes(Species ~ ., data=iris)
m_dt   <- rpart(Species ~ ., data=iris, method="class")
m_rf   <- randomForest(Species ~ ., data=iris, ntree=100)
m_svm  <- svm(Species ~ ., data=iris, probability=TRUE)
m_nn   <- nnet(Species ~ ., data=iris, size=5, maxit=500, trace=FALSE)


---
### Aufgabe 4: **Prognosen erzeugen**
Befehl `predict`

In [ ]:

# Vorhersagen
pred_log <- predict(m_log, iris)
pred_lda <- predict(m_lda, iris)$class
pred_nb  <- predict(m_nb, iris)
pred_dt  <- predict(m_dt, iris, type="class")
pred_rf  <- predict(m_rf, iris)
pred_svm <- predict(m_svm, iris)
pred_nn  <- predict(m_nn, iris, type="class")


---
### Aufgabe 5: **Fehlermetriken**

- Accuracy: Anteil richtiger Prognosen (`ACC`)
- False positive für eine bestimmte Klasse (`FP`)
- False negative für eine bestimmte Klasse (`FN`)


In [ ]:

# Fehlermetriken
acc <- function(y, yh) mean(y == yh)
fp  <- function(y, yh) {y0=unique(y)[2];   mean(y == y0 & yh != y0)}
fn  <- function(y, yh) {y0=unique(y)[2];   mean(y != y0 & yh == y0)}


---
### Aufgabe 6: **Ergebnisse berechnen**

In [ ]:

# Ergebnisse berechnen
models <- list(
  LOG=pred_log, LDA=pred_lda, NB=pred_nb, DT=pred_dt, RF=pred_rf, SVM=pred_svm, NN=pred_nn)

df_res <- data.frame(
  ACC  = sapply(models, function(p) acc(iris$Species, p)),
  FP   = sapply(models, function(p) fp(iris$Species, p)),
  FN   = sapply(models, function(p) fn(iris$Species, p)))

print(round(df_res,3))


      ACC    FP    FN
LOG 0.987 0.007 0.007
LDA 0.980 0.013 0.007
NB  0.960 0.020 0.020
DT  0.960 0.007 0.033
RF  1.000 0.000 0.000
SVM 0.973 0.013 0.013
NN  0.993 0.007 0.000


---
### Aufgabe 7: **Ergebnisse graphisch darstellen**

In [ ]:

# Barplots
par(mar=c(8,4,8,4));par(mfrow=c(1,1))
barplot(df_res$ACC, names=row.names(df_res), col="green", main="Accuracy: Iris-Daten")
par(mar=c(3,4,3,4));par(mfrow=c(2,1))
barplot(df_res$FP,  names=row.names(df_res), col="tomato", main=paste("False Positives - Class:",unique(iris$Species)[2]))
barplot(df_res$FN,  names=row.names(df_res), col="steelblue", main=paste("False Negatives - Class:",unique(iris$Species)[2]))


---
### Zusätzliche Aufgaben: **Anwendung an unterschiedlichen Datensätzen**

---
#### **Generische Klassifizierungsfunktion**

Zur Analyse belieger Datensätze

In [ ]:

analyze_dataset <- function(df, target_col, dataset_name="Dataset", scale_features=TRUE, critic_class=2, train_frac=0.8, seed=1){
  set.seed(seed)

  # NA entfernen
  df <- na.omit(df)

  # Nur numerische Features
  X <- df[, sapply(df, is.numeric)]
  Y <- df[[target_col]]

  # Skalierung optional
  if(scale_features) X <- as.data.frame(scale(X))

  # PCA und Scatterplot
  pca <- prcomp(X, scale.=FALSE)
  df_pca <- cbind(as.data.frame(pca$x[,1:2]), Class=Y)
  par(mfrow=c(2,2))
  plot(pca$x[,1:2], col=as.numeric(Y)+1, main= paste("PCA Scatterplot:", dataset_name))
  legend("bottomright", legend=unique(Y), col=c(2,3,4), pch=1, bty='n', cex=.7)

  # Train/Test-Datenaufteilung (Kreuzvalidierung)
  id <- sample(1:nrow(df), train_frac*nrow(df))
  train <- cbind(X[id, ], Y=Y[id])
  test  <- cbind(X[-id, ], Y=Y[-id])

  # Modelle trainieren
  m_log <- multinom(Y ~ ., data=train, trace=FALSE)
  m_lda <- lda(Y ~ ., data=train)
  m_nb  <- naiveBayes(Y ~ ., data=train)
  m_dt  <- rpart(Y ~ ., data=train, method="class")
  m_rf  <- randomForest(Y ~ ., data=train, ntree=100)
  m_svm <- svm(Y ~ ., data=train, probability=TRUE, scale=FALSE)
  m_nn  <- nnet(Y ~ ., data=train, size=5, maxit=500, trace=FALSE)

  # Vorhersagen
  pred_log <- predict(m_log, test)
  pred_lda <- predict(m_lda, test)$class
  pred_nb  <- predict(m_nb, test)
  pred_dt  <- predict(m_dt, test, type="class")
  pred_rf  <- predict(m_rf, test)
  pred_svm <- predict(m_svm, test)
  pred_nn  <- predict(m_nn, test, type="class")

  # Fehlermetriken
  acc <- function(y, yh) mean(y == yh)
  fp  <- function(y, yh) {y0=unique(y)[critic_class];   mean(y == y0 & yh != y0)}
  fn  <- function(y, yh) {y0=unique(y)[critic_class];   mean(y != y0 & yh == y0)}

  # Ergebnisse
  models <- list(LOG=pred_log, LDA=pred_lda, NB=pred_nb, DT=pred_dt, RF=pred_rf, SVM=pred_svm, NN=pred_nn)
  df_res <- data.frame(
    ACC   = sapply(models, function(p) acc(test$Y, p)),
    FP    = sapply(models, function(p) fp(test$Y, p)),
    FN    = sapply(models, function(p) fn(test$Y, p)))
  print(round(df_res,3))

  # Barplots
  barplot(df_res$ACC, names.arg=row.names(df_res), col="green", main=paste("Accuracy:", dataset_name), ylim=c(0,1))
  barplot(df_res$FP, names.arg=row.names(df_res), col="tomato", main=paste("False Positives - Class:",unique(Y)[critic_class]), ylim=c(0,max(.2,df_res$FP)))
  barplot(df_res$FN, names.arg=row.names(df_res), col="steelblue", main=paste("False Negatives - Class:",unique(Y)[critic_class]), ylim=c(0,max(.2,df_res$FN)))

  return(df_res)
}


---
#### **Penguins-Datensatz**
Der Penguins-Datensatz enthält biometrische Messungen verschiedener Pinguinarten, wie Schnabel- und Flügellängen sowie Gewicht, um Arten anhand dieser Merkmale zu unterscheiden.

In [ ]:

# Penguins-Datensatz
data(penguins)
summary(penguins)
results_penguins <- analyze_dataset(penguins, "species", "Penguins", TRUE, 3)


---
#### **Sonar-Datensatz**
Der Sonar-Datensatz enthält Reflektionsmessungen von Sonarsignalen, um zwischen Metallzylindern und Felsen anhand numerischer Merkmale zu unterscheiden.

In [ ]:

# Sonar-Dartensatz
data(Sonar)
summary(Sonar)
results_sonar <- analyze_dataset(Sonar, "Class", "Sonar")


---
#### **Glass-Datensatz**
Der Glass-Datensatz umfasst chemische Zusammensetzungen verschiedener Glasarten, um deren Typen anhand numerischer Merkmale zu klassifizieren.

In [ ]:

# Glass-Datensatz
data(Glass)
summary(Glass)
results_glass <- analyze_dataset(Glass, "Type", "Glass")


---
#### **Vowel**
Der Vowel-Datensatz enthält akustische Merkmale gesprochener Vokale, die zur Unterscheidung der Vokaltypen verwendet werden

In [ ]:

# Vowel-Datensatz
data("Vowel")
summary(Vowel)
results_vowel <- analyze_dataset(Vowel, "Class", "Vowel Recognition")